In [8]:
import scipy as scp
import numpy as np
import time
from datetime import timedelta



print("hello world")


hello world


In [26]:
#class gb():
start_time = time.time()
end_time = 0
def debug(level='partial'):
    match level:
        case 'partial':
            print('   DEBUG LEVEL PARTIAL')
            print("Iteration {} at current time = {}".format(stepN, t))
            printAlligned('S position',Spos)
            printAlligned('S velocity', Svel)
            printAlligned('S acceleration', Sacc)
        case 'full':
            print('   DEBUG LEVEL FULL')
            print("List of globals:")
            print("Iteration {} at current time = {}".format(stepN, t))
            printAlligned('S position',Spos)
            printAlligned('S velocity', Svel)
            printAlligned('S acceleration', Sacc)
        case 'start':
            print('   DEBUG LEVEL START')
            print("Start time = {}".format(start_time))
            printAlligned('S position',Spos)
            printAlligned('S velocity', Svel)
            printAlligned('S acceleration', Sacc)
        case 'end':
            print('   DEBUG LEVEL END')
            end_time = time.time()
            print("Wall runtime = {}".format(timedelta(end_time-start_time)))
def initialize(inSpos,inSvel,stepsize=[1,1],startdebuglevel='start'):
    # Constants
    global G0, d, Me, Mm, Mt, Re, Rm, P, aSt, aDy, stepN, t
    G0 = 6.6726e-11
    d = 3.844e5
    Me = 5.9742e24
    Mm = 7.35e22
    Mt = Me + Mm
    Re = d*Mm/Mt
    Rm = d*Me/Mt
    P = np.sqrt(4 * np.pi**2 * (d*1000)**3 / G0 / Mt)
    aSt = stepsize[0]
    

    # Variables
    aDy = stepsize[1]
    stepN = 0
    t = 0

    # Vectors
    global Spos, Svel, Sacc

    Epos = np.array([-Re,0])

    Mpos = np.array([Rm,0])

    Spos = np.array(inSpos)
    Svel = np.array(inSvel)
    Sacc = np.zeros(2)
    debug(startdebuglevel)
initialize([1.0,2.0],[4.0,5.0])
def printAlligned(message,variable):
    output = print("{:20s}= {}".format(message, variable))
    return output


print(start_time)
printAlligned('S position',Spos)
print("{:20s}= {}".format('S position',Spos))
print("S velocity     = {}".format(Svel))
print("S acceleration = {}".format(Sacc))
print("P = {}".format(P))
print("HR P = {}".format(timedelta(seconds = P)))

def Epos(time):
    output = np.array([-Re*np.cos(time/P),-Re*np.sin(time/P)])
    return output
def Mpos(time):
    output = np.array([Rm*np.cos(time/P),Rm*np.sin(time/P)])
    return output
def integrateStep(upper, stop):
    return None
print(globals)

   DEBUG LEVEL START
Start time = 1697126100.4413605
S position          = [1. 2.]
S velocity          = [4. 5.]
S acceleration      = [0. 0.]
1697126100.4413605
S position          = [1. 2.]
S position          = [1. 2.]
S velocity     = [4. 5.]
S acceleration = [0. 0.]
P = 2357285.970587522
HR P = 27 days, 6:48:05.970588
<built-in function globals>


In [18]:
a = 2357285.970587522
b = timedelta(seconds = a)
print(b)

27 days, 6:48:05.970588
